# Examine if the important mobility ties are structural

- Combine three networks: Spatial + (strong or insig weak) + (sig weak pos or neg).
- Compute the edge_betweenness_centralities
- Double think: maybe it is fine to combine only two networks: spatial + others. 

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
from scipy.sparse import csr_matrix


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [29]:
import time

In [2]:
from sklearn.preprocessing import normalize

In [3]:
import sys
sys.path.append("../")
import utils

In [4]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [5]:
# read files
with open("../../data/02_intermediate/boston_stays.pickle", 'rb') as f:
    df_boston = pickle.load(f)

with open("../../data/02_intermediate/miami_stays.pickle", 'rb') as f:
    df_miami = pickle.load(f)

with open("../../data/02_intermediate/chicago_stays.pickle", 'rb') as f:
    df_chicago = pickle.load(f)

with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_activity_unweighted_dic = pickle.load(f)
    
with open("../../data/03_processed/A_home_activity_three_cities_weighted_dic.pickle", 'rb') as f:
    A_home_activity_weighted_dic = pickle.load(f)


In [6]:
with open("../../data/03_processed/spatial_network_boston_miami_chicago_dic.pickle", 'rb') as f:
    spatial_network_dic = pickle.load(f)


In [7]:
# read shapefiles
with open("../../data/02_intermediate/boston_miami_chicago_ct_shp_dic.pickle", 'rb') as f:
    shp_dic = pickle.load(f)


In [8]:
# read evaluation files
with open("../../data/05_model_outputs/lasso_coefficients.pickle", 'rb') as f:
    lasso_coef = pickle.load(f)
    

In [9]:
# activity counts for the three cities
# boston
activity_counts_boston = np.unique(df_boston.cat, return_counts = True)

# miami
activity_counts_miami = np.unique(df_miami.cat, return_counts = True)

# chicago
activity_counts_chicago = np.unique(df_chicago.cat, return_counts = True)

# convert the counts to df
activity_counts_dic = {}
activity_counts_dic['boston']=activity_counts_boston
activity_counts_dic['miami']=activity_counts_miami
activity_counts_dic['chicago']=activity_counts_chicago

# turn them to dataframes
activity_counts_df_dic = {}
for key_ in activity_counts_dic.keys():
    activity_counts = activity_counts_dic[key_]
    activity_count_df = pd.DataFrame(activity_counts[1],
                                 index = activity_counts[0],
                                 columns = ['count'])
    sorted_activity_count_df = activity_count_df.sort_values('count', ascending=False)
    activity_counts_df_dic[key_] = sorted_activity_count_df


In [10]:
shp_dic['boston']

,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
GEOID,,,,,,,,,,,,
25009201100,25,009,201100,2011,Census Tract 2011,G5020,S,2786306,3481257,+42.4282342,-070.9260711,"POLYGON ((-70.94328 42.43815, -70.94324 42.438..."
25009202101,25,009,202101,2021.01,Census Tract 2021.01,G5020,S,1454942,7226,+42.4758943,-070.9085318,"POLYGON ((-70.92295 42.46878, -70.92292 42.468..."
25009202102,25,009,202102,2021.02,Census Tract 2021.02,G5020,S,2919051,24048,+42.4829911,-070.9199549,"POLYGON ((-70.93398 42.49080, -70.92844 42.489..."
25009202200,25,009,202200,2022,Census Tract 2022,G5020,S,3452798,1441782,+42.4695768,-070.8975175,"POLYGON ((-70.92226 42.46741, -70.92180 42.467..."
25009203100,25,009,203100,2031,Census Tract 2031,G5020,S,3720285,1481336,+42.4871332,-070.8724094,"POLYGON ((-70.89847 42.48483, -70.89828 42.484..."
...,...,...,...,...,...,...,...,...,...,...,...,...
33017085000,33,017,085000,850,Census Tract 850,G5020,S,120890933,5035519,+43.2155657,-071.0389797,"POLYGON ((-71.13352 43.20372, -71.13341 43.203..."
33017086000,33,017,086000,860,Census Tract 860,G5020,S,126932596,5722104,+43.2753082,-071.1700041,"POLYGON ((-71.24732 43.27528, -71.24721 43.275..."
33017087000,33,017,087000,870,Census Tract 870,G5020,S,94758837,909784,+43.3669235,-071.0625029,"POLYGON ((-71.15923 43.35524, -71.15892 43.355..."


### Part 1. Compute edge bet centralities for three graphs (each iteration)

- Part 1. Combine spatial + strong or weak insig + weak pos or neg networks
- Reasoning: we compare the significant mobility networks to other two simultaneously.
- However, it seems that we could compare the four mobility networks separately.


In [11]:
# total time: about 2 hours.
# init
network_property_dic = {}

# fixed parameters
spatial_net_name = 'queen_contiguity_adj_df'
model_type = 'lasso (no socio-demographics)'
threshold = 1.0
top_K_as_strong_mobility_ties = 50
sampling_size = 10 # number of samples we need from each activity list (strong, weak insig, etc.)


# five layers of iteration. It is slow...
for city in ['boston','chicago','miami']:
    network_property_dic[city] = {}

    # need to try and test if the spatial net is connected. 
    # if not, use only the largest component for the analysis.     
    spatial_net = spatial_network_dic[city][spatial_net_name]
    G_spatial = nx.from_pandas_adjacency(spatial_net)

    if nx.number_connected_components(G_spatial) > 1:
        # if city = chicago or miami, then the network is disconnected. We choose the giant component. 
        # find the giant component
        Gcc = sorted(nx.connected_components(G_spatial), key=len, reverse=True)
        for G_sub in Gcc:
            print(len(G_sub)) # print the size of the components.
        G0 = G_spatial.subgraph(Gcc[0])
        giant_component_node_list = sorted(list(G0.nodes))
        giant_component_node_list

        # replace the input shapefile and the spatial networks
        spatial_network_dic[city][spatial_net_name] = spatial_network_dic[city][spatial_net_name].loc[giant_component_node_list, giant_component_node_list]
        shp_dic[city] = shp_dic[city].loc[giant_component_node_list, :]

        # recreate the spatial net and spatial graph
        spatial_net = spatial_network_dic[city][spatial_net_name]
        G_spatial = nx.from_pandas_adjacency(spatial_net)

    print(city)
    print("Baseline average distance: ", nx.average_shortest_path_length(G_spatial))

    for output_var in ['inc_median_household_2018', 'property_value_median_2018', 'rent_median_2018']:
        network_property_dic[city][output_var] = {}

        # create four lists of activities: strong, weak insig, weak sig pos, weak sig neg.
        strong_activity_list = list(activity_counts_df_dic[city].index[:top_K_as_strong_mobility_ties]) 
        weak_sig_activity_list = list(lasso_coef[city][output_var][model_type].index)
        weak_sig_neg_activities = list(lasso_coef[city][output_var][model_type]['value'].loc[lasso_coef[city][output_var][model_type]['value'] < 0.0].index)
        weak_sig_pos_activities = list(lasso_coef[city][output_var][model_type]['value'].loc[lasso_coef[city][output_var][model_type]['value'] > 0.0].index)
        weak_insig_activity_list = list(set(activity_counts_df_dic[city].index).difference(set(strong_activity_list)).difference(set(weak_sig_activity_list)))

        # 
        activity_type_list = ['strong', 'weak_sig_neg', 'weak_sig_pos', 'weak_insig']
        activity_list_dic = {}
        activity_list_dic['strong'] = strong_activity_list
        activity_list_dic['weak_sig_neg'] = weak_sig_neg_activities
        activity_list_dic['weak_sig_pos'] = weak_sig_pos_activities
        activity_list_dic['weak_insig'] = weak_insig_activity_list

        # combine spatial, benchmark, and target networks to compute the edge_bet_centralities.

        for activity_type_benchmark in ['strong', 'weak_insig']:
            for activity_type_target in ['weak_sig_neg', 'weak_sig_pos']:
                print(activity_type_benchmark, activity_type_target)
                network_property_dic[city][output_var][(activity_type_benchmark, activity_type_target)] = {}

                activity_type_benchmark_list = activity_list_dic[activity_type_benchmark]
                activity_type_target_list = activity_list_dic[activity_type_target]

                for i in range(sampling_size):
                    activity_benchmark_name = np.random.choice(activity_type_benchmark_list)
                    activity_target_name = np.random.choice(activity_type_target_list)
                    print(activity_benchmark_name, activity_target_name)

                    spatial_net = spatial_network_dic[city][spatial_net_name]
                    mobility_benchmark_net = utils.turn_df_to_adj(A_home_activity_unweighted_dic[city][threshold][activity_benchmark_name], shp_dic[city])
                    mobility_target_net = utils.turn_df_to_adj(A_home_activity_unweighted_dic[city][threshold][activity_target_name], shp_dic[city])

                    # integrate networks and compute betweenness centralities 
                    integrated_adj = spatial_net.add(mobility_benchmark_net, fill_value = 0.0).add(mobility_target_net, fill_value = 0.0)
                    integrated_adj.values[integrated_adj.values > 1.0] = 1.0 # valid adj matrices

                    # Get the edge betweenness metrics
                    G = nx.from_pandas_adjacency(integrated_adj)
                    edge_bet_centrality_graph = nx.edge_betweenness_centrality(G) # joint centrality metrics

                    # turn the graph info to dataframe
                    edge_bet_centrality_df = pd.DataFrame(edge_bet_centrality_graph.values(), 
                                                                           index = list(edge_bet_centrality_graph.keys()), 
                                                                           columns = ['edge_bet_centrality'])


                    # separate the spatial, strong, and weak ties
                    G_spatial = nx.from_pandas_adjacency(spatial_net)
                    G_mobility_benchmark = nx.from_pandas_adjacency(mobility_benchmark_net)
                    G_mobility_target = nx.from_pandas_adjacency(mobility_target_net)

                    spatial_edges = list(G_spatial.edges())
                    mobility_benchmark_edges = list(G_mobility_benchmark.edges())
                    mobility_target_edges = list(G_mobility_target.edges())

                    # Boston - income - four types of activityes - specific edge bet centralities
                    network_property_dic[city][output_var][(activity_type_benchmark, activity_type_target)][('spatial', activity_benchmark_name, activity_target_name)]=(edge_bet_centrality_df.loc[spatial_edges, 'edge_bet_centrality'].mean(),
                                                                                                       edge_bet_centrality_df.loc[mobility_benchmark_edges, 'edge_bet_centrality'].mean(), 
                                                                                                       edge_bet_centrality_df.loc[mobility_target_edges, 'edge_bet_centrality'].mean())


                    
                    
                    

boston
Baseline average distance:  13.718757120536441
strong weak_sig_neg
Italian Brazilian
Residence Hall Latin American
Tech Startup Caribbean
Hospital Fried Chicken
Grocery Store Caribbean
Road Laundromat
Salon / Barbershop Caribbean
Residence Hall Fried Chicken
Laundry Laundromat
Residential Caribbean
strong weak_sig_pos
High School Tennis Court
Pool French
Office Fishing Store
Donuts Tennis Court
Pizza Fishing Store
Sandwiches Skating Rink
Park Skating Rink
Church Food Stand
Office Science Museum
Supermarket French
weak_insig weak_sig_neg
Museum Fried Chicken
Gate Brazilian
Trail Latin American
Ethiopian Latin American
Frat House Fried Chicken
Prayer Room Brazilian
Polish Latin American
Beach Laundromat
Health & Beauty Brazilian
Transportation Services Laundromat
weak_insig weak_sig_pos
Language School Science Museum
Beach Bar French
Video Games Tennis Court
Street Art Food Stand
Noodles Skating Rink
Taiwanese Football
Corporate Amenity Tennis Court
Indian Chinese Fishing Store
Ca

Disc Golf Synagogue
strong weak_sig_neg
Dentist's Office Lighthouse
Gas Station Gay Bar
Residential Gay Bar
Office Gay Bar
Bar Pawn Shop
Beach Discount Store
Shop Lighthouse
Event Space Lighthouse
Gas Station Lighthouse
Neighborhood Discount Store
strong weak_sig_pos
Sandwiches Salad
Furniture / Home Circus
Bank University
Harbor / Marina Track
Government Golf Course
Pizza Sushi
Lounge University
Italian Basketball
Bank Golf Course
Grocery Store Cantonese
weak_insig weak_sig_neg
German Pawn Shop
Erotic Museum Lighthouse
Night Market Gay Bar
Garden Center Pawn Shop
Cable Car Discount Store
Home Services Discount Store
Travel Lighthouse
Travel Agency Pawn Shop
Hot Spring Pawn Shop
Post Office Lighthouse
weak_insig weak_sig_pos
Gun Range Circus
Dive Bar Salad
Gourmet Basketball
IT Services University
Fish Market Track
German University
Tram Track
Fishing Spot Circus
Outdoor Event Space Basketball
Bridal University


## save

In [12]:
with open('../../data/05_model_outputs/network_property_edge_bet_centrality.pickle', 'wb') as f:
    pickle.dump(network_property_dic, f)


## open

In [13]:
with open('../../data/05_model_outputs/network_property_edge_bet_centrality.pickle', 'rb') as f:
    network_property_dic = pickle.load(f)

In [15]:
network_property_dic['boston']['inc_median_household_2018']

{('strong',
  'weak_sig_neg'): {('spatial', 'Italian', 'Brazilian'): (4.90988798752745e-05,
   0.00011468902324290533,
   9.882343085783116e-05), ('spatial',
   'Residence Hall',
   'Latin American'): (0.00022458565992399394, 0.0005740032990235093, 0.000381001077884699), ('spatial',
   'Tech Startup',
   'Caribbean'): (9.614840655075186e-05,
   0.0002495983075323691,
   0.00014291645271266538), ('spatial',
   'Hospital',
   'Fried Chicken'): (5.349496610079905e-05, 0.00019465617313739965, 6.395696346199216e-05), ('spatial',
   'Grocery Store',
   'Caribbean'): (6.088677214268247e-05,
   0.00011878760016217698,
   8.633763464931991e-05), ('spatial',
   'Road',
   'Laundromat'): (6.780755821001826e-05, 0.0001394879430019811, 8.948370708462066e-05), ('spatial',
   'Salon / Barbershop',
   'Caribbean'): (4.714324213718289e-05,
   7.613246424278207e-05,
   6.58241459703407e-05), ('spatial',
   'Residence Hall',
   'Fried Chicken'): (0.00022075568582207109, 0.0005897393556411182, 0.000437632

### Part 2. Compute edge bet centralities for two graphs (each iteration)

- Combine spatial + strong or weak insig or weak pos or neg networks


In [28]:
A_home_activity_unweighted_dic[city][threshold].keys()

dict_keys(['ATM', 'Academic Building', 'Accessories', 'Acupuncturist', 'Administrative Building', 'Adult Boutique', 'Adult Education Center', 'Advertising Agency', 'Afghan', 'African', 'Airport', 'Airport Service', 'Alternative Healer', 'American', 'Amphitheater', 'Animal Shelter', 'Antiques', 'Apparel', 'Apres Ski Bar', 'Aquarium', 'Arcade', 'Arepas', 'Argentinian', 'Art Gallery', 'Art Museum', 'Art Studio', 'Arts', 'Arts & Crafts', 'Arts & Entertainment', 'Asian', 'Assisted Living', 'Astrologer', 'Athletics & Sports', 'Auditorium', 'Australian', 'Auto Dealer', 'Auto Garage', 'Auto Workshop', 'Automotive', 'B & B', 'BBQ', 'Baby Store', 'Bagels', 'Baggage Claim', 'Bakery', 'Ballroom', 'Bank', 'Bar', 'Baseball', 'Baseball Field', 'Basketball', 'Basketball Court', 'Bath House', 'Bathing Area', 'Bay', 'Beach', 'Beach Bar', 'Beer Bar', 'Beer Garden', 'Beer Store', 'Belgian', 'Big Box Store', 'Bike', 'Bike Shop', 'Bike Trail', 'Billiards', 'Bistro', 'Board Shop', 'Boarding', 'Boat / Ferry',

In [34]:
# time
beginning_time = time.time()

# init
network_property_edge_centrality_dic = {}

# fixed parameters
spatial_net_name = 'queen_contiguity_adj_df'
model_type = 'lasso (no socio-demographics)'
threshold = 1.0
top_K_as_strong_mobility_ties = 50
# sampling_size = 10 # number of samples we need from each activity list (strong, weak insig, etc.)


# five layers of iteration. It is slow...
for city in ['boston','chicago','miami']:
    network_property_edge_centrality_dic[city] = {}

    # need to try and test if the spatial net is connected. 
    # if not, use only the largest component for the analysis.     
    spatial_net = spatial_network_dic[city][spatial_net_name]
    G_spatial = nx.from_pandas_adjacency(spatial_net)

    if nx.number_connected_components(G_spatial) > 1:
        # if city = chicago or miami, then the network is disconnected. We choose the giant component. 
        # find the giant component
        Gcc = sorted(nx.connected_components(G_spatial), key=len, reverse=True)
        for G_sub in Gcc:
            print(len(G_sub)) # print the size of the components.
        G0 = G_spatial.subgraph(Gcc[0])
        giant_component_node_list = sorted(list(G0.nodes))
        giant_component_node_list

        # replace the input shapefile and the spatial networks
        spatial_network_dic[city][spatial_net_name] = spatial_network_dic[city][spatial_net_name].loc[giant_component_node_list, giant_component_node_list]
        shp_dic[city] = shp_dic[city].loc[giant_component_node_list, :]

        # recreate the spatial net and spatial graph
        spatial_net = spatial_network_dic[city][spatial_net_name]
        G_spatial = nx.from_pandas_adjacency(spatial_net)

    print(city)
    print("Baseline average distance: ", nx.average_shortest_path_length(G_spatial))

    for idx in range(len(list(A_home_activity_unweighted_dic[city][threshold].keys()))):
        activity_name = list(A_home_activity_unweighted_dic[city][threshold].keys())[idx]
        network_property_edge_centrality_dic[city][activity_name] = {}
        current_time = time.time()
        elapse_time = current_time - beginning_time
        print(idx, activity_name, elapse_time/60.0, "minutes", end = '\r')
        
        spatial_net = spatial_network_dic[city][spatial_net_name]
        mobility_net = utils.turn_df_to_adj(A_home_activity_unweighted_dic[city][threshold][activity_name], shp_dic[city])
    
        # integrate networks and compute betweenness centralities 
        integrated_adj = spatial_net.add(mobility_net, fill_value = 0.0)
        integrated_adj.values[integrated_adj.values > 1.0] = 1.0 # valid adj matrices
    
        # Get the edge betweenness metrics
        G = nx.from_pandas_adjacency(integrated_adj)
        edge_bet_centrality_graph = nx.edge_betweenness_centrality(G) # joint centrality metrics

        # turn the graph info to dataframe
        edge_bet_centrality_df = pd.DataFrame(edge_bet_centrality_graph.values(), 
                                                               index = list(edge_bet_centrality_graph.keys()), 
                                                               columns = ['edge_bet_centrality'])

        # separate the spatial, strong, and weak ties
        G_spatial = nx.from_pandas_adjacency(spatial_net)
        G_mobility = nx.from_pandas_adjacency(mobility_net)
    
        spatial_edges = list(G_spatial.edges())
        mobility_edges = list(G_mobility.edges())
    
        # Boston - income - four types of activityes - specific edge bet centralities
        network_property_edge_centrality_dic[city][activity_name][('spatial', activity_name)]=(edge_bet_centrality_df.loc[spatial_edges, 'edge_bet_centrality'].mean(),
                                                                                               edge_bet_centrality_df.loc[mobility_edges, 'edge_bet_centrality'].mean())




boston
Baseline average distance:  13.718757120536441
chicago Exhibit 39.33925555149714 minutesesutessnutes
Baseline average distance:  15.919317195904911
miamioo Exhibit 224.02720179955165 minutesesnutesutesss
Baseline average distance:  25.13196220207745


### save

In [35]:
with open('../../data/05_model_outputs/network_property_edge_bet_centrality_simpler.pickle', 'wb') as f:
    pickle.dump(network_property_edge_centrality_dic, f)


### Analysis

In [16]:
network_property_dic[city][output_var].keys()

dict_keys([('strong', 'weak_sig_neg'), ('strong', 'weak_sig_pos'), ('weak_insig', 'weak_sig_neg'), ('weak_insig', 'weak_sig_pos')])

In [17]:
network_property_dic[city][output_var][('strong', 'weak_sig_neg')]

{('spatial', "Dentist's Office", 'Lighthouse'): (5.183488220953141e-05,
  9.249421570993838e-05,
  7.155482175600366e-05),
 ('spatial', 'Gas Station', 'Gay Bar'): (3.5554767952653426e-05,
  4.960937683379324e-05,
  5.319450692891655e-05),
 ('spatial', 'Residential', 'Gay Bar'): (2.6226043164810372e-05,
  4.1844597944195725e-05,
  4.3232248589001863e-05),
 ('spatial', 'Office', 'Gay Bar'): (1.9725956373145664e-05,
  3.427791317736445e-05,
  3.037960519715993e-05),
 ('spatial', 'Bar', 'Pawn Shop'): (3.191894574525126e-05,
  8.614868409442068e-05,
  4.5777158644797446e-05),
 ('spatial', 'Beach', 'Discount Store'): (3.432399643085734e-05,
  0.00012514087666528603,
  4.6383597520469626e-05),
 ('spatial', 'Shop', 'Lighthouse'): (4.532094636219517e-05,
  9.631485019310046e-05,
  6.242518594635883e-05),
 ('spatial', 'Event Space', 'Lighthouse'): (6.347474656306515e-05,
  0.0001544307503759619,
  7.731605191949097e-05),
 ('spatial', 'Gas Station', 'Lighthouse'): (3.746727150707587e-05,
  5.1954

In [18]:
# Check
city = 'boston'
output_var = 'inc_median_household_2018'

edge_bet_df_dic = {}

for activity_tuple in list(network_property_dic[city][output_var].keys()):
    print(activity_tuple)
    edge_bet_df_dic[activity_tuple] = pd.DataFrame(network_property_dic[city][output_var][activity_tuple].values(), 
                                                  columns = ['spatial', activity_tuple[0], activity_tuple[1]],
                                                  index = network_property_dic[city][output_var][activity_tuple].keys())
    # normalize by using the spatial column (Q: Is this approach correct?)
    edge_bet_df_dic[activity_tuple][activity_tuple[0]] = edge_bet_df_dic[activity_tuple][activity_tuple[0]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple][activity_tuple[1]] = edge_bet_df_dic[activity_tuple][activity_tuple[1]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple]['spatial'] = np.ones(edge_bet_df_dic[activity_tuple].shape[0])
#     edge_bet_df_dic[activity_type]['spatial'] = edge_bet_df_dic[activity_type][activity_type]/edge_bet_df_dic[activity_type]['spatial']
    # print
    print(edge_bet_df_dic[activity_tuple].describe())

# edge_bet_df_dic
    

('strong', 'weak_sig_neg')
       spatial     strong  weak_sig_neg
count     10.0  10.000000     10.000000
mean       1.0   2.329490      1.544095
std        0.0   0.592095      0.271618
min        1.0   1.614918      1.195570
25%        1.0   1.977498      1.401695
50%        1.0   2.280292      1.453907
75%        1.0   2.585935      1.650347
max        1.0   3.638776      2.012743
('strong', 'weak_sig_pos')
       spatial     strong  weak_sig_pos
count     10.0  10.000000     10.000000
mean       1.0   1.987773      2.083930
std        0.0   0.354734      1.040939
min        1.0   1.595069      0.736750
25%        1.0   1.747808      1.505313
50%        1.0   1.920208      1.769120
75%        1.0   2.023239      2.674499
max        1.0   2.612533      4.183630
('weak_insig', 'weak_sig_neg')
       spatial  weak_insig  weak_sig_neg
count     10.0   10.000000     10.000000
mean       1.0    6.461242      2.587955
std        0.0   11.886640      0.424949
min        1.0    0.844192     

In [19]:
# Check
city = 'boston'
output_var = 'property_value_median_2018'

edge_bet_df_dic = {}

for activity_tuple in list(network_property_dic[city][output_var].keys()):
    print(activity_tuple)
    edge_bet_df_dic[activity_tuple] = pd.DataFrame(network_property_dic[city][output_var][activity_tuple].values(), 
                                                  columns = ['spatial', activity_tuple[0], activity_tuple[1]],
                                                  index = network_property_dic[city][output_var][activity_tuple].keys())
    # normalize by using the spatial column (Q: Is this approach correct?)
    edge_bet_df_dic[activity_tuple][activity_tuple[0]] = edge_bet_df_dic[activity_tuple][activity_tuple[0]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple][activity_tuple[1]] = edge_bet_df_dic[activity_tuple][activity_tuple[1]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple]['spatial'] = np.ones(edge_bet_df_dic[activity_tuple].shape[0])
#     edge_bet_df_dic[activity_type]['spatial'] = edge_bet_df_dic[activity_type][activity_type]/edge_bet_df_dic[activity_type]['spatial']
    # print
    print(edge_bet_df_dic[activity_tuple].describe())

# edge_bet_df_dic
    

('strong', 'weak_sig_neg')
       spatial     strong  weak_sig_neg
count     10.0  10.000000     10.000000
mean       1.0   1.933361      1.857445
std        0.0   0.345337      0.495024
min        1.0   1.469699      1.323859
25%        1.0   1.678012      1.531265
50%        1.0   1.835386      1.643277
75%        1.0   2.192186      2.225620
max        1.0   2.465374      2.724171
('strong', 'weak_sig_pos')
       spatial     strong  weak_sig_pos
count     10.0  10.000000     10.000000
mean       1.0   2.245996      2.433182
std        0.0   0.789715      1.276990
min        1.0   1.580810      1.130688
25%        1.0   1.656888      1.837064
50%        1.0   1.982973      2.122442
75%        1.0   2.327066      2.490496
max        1.0   3.673577      5.803658
('weak_insig', 'weak_sig_neg')
       spatial  weak_insig  weak_sig_neg
count     10.0   10.000000     10.000000
mean       1.0    2.878003      2.719718
std        0.0    1.021859      0.495237
min        1.0    1.747724     

In [20]:
# Check
city = 'boston'
output_var = 'rent_median_2018'

edge_bet_df_dic = {}

for activity_tuple in list(network_property_dic[city][output_var].keys()):
    print(activity_tuple)
    edge_bet_df_dic[activity_tuple] = pd.DataFrame(network_property_dic[city][output_var][activity_tuple].values(), 
                                                  columns = ['spatial', activity_tuple[0], activity_tuple[1]],
                                                  index = network_property_dic[city][output_var][activity_tuple].keys())
    # normalize by using the spatial column (Q: Is this approach correct?)
    edge_bet_df_dic[activity_tuple][activity_tuple[0]] = edge_bet_df_dic[activity_tuple][activity_tuple[0]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple][activity_tuple[1]] = edge_bet_df_dic[activity_tuple][activity_tuple[1]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple]['spatial'] = np.ones(edge_bet_df_dic[activity_tuple].shape[0])
#     edge_bet_df_dic[activity_type]['spatial'] = edge_bet_df_dic[activity_type][activity_type]/edge_bet_df_dic[activity_type]['spatial']
    # print
    print(edge_bet_df_dic[activity_tuple].describe())

# edge_bet_df_dic
    

('strong', 'weak_sig_neg')
       spatial     strong  weak_sig_neg
count     10.0  10.000000     10.000000
mean       1.0   2.399010      1.506980
std        0.0   0.393740      0.203280
min        1.0   1.965711      1.274883
25%        1.0   2.185647      1.333280
50%        1.0   2.243746      1.497723
75%        1.0   2.589560      1.600721
max        1.0   3.267437      1.922552
('strong', 'weak_sig_pos')
       spatial     strong  weak_sig_pos
count     10.0  10.000000     10.000000
mean       1.0   2.306325      2.077127
std        0.0   0.601383      0.842560
min        1.0   1.585911      1.318057
25%        1.0   1.914332      1.478640
50%        1.0   2.107861      1.772733
75%        1.0   2.628730      2.470564
max        1.0   3.672068      3.999264
('weak_insig', 'weak_sig_neg')
       spatial  weak_insig  weak_sig_neg
count     10.0   10.000000     10.000000
mean       1.0    3.302278      2.758449
std        0.0    0.543152      0.869340
min        1.0    2.480772     

In [26]:
# Check
city = 'miami'
# output_var = 'inc_median_household_2018'
# output_var = 'property_value_median_2018'
output_var = 'rent_median_2018'

edge_bet_df_dic = {}

for activity_tuple in list(network_property_dic[city][output_var].keys()):
    print(activity_tuple)
    edge_bet_df_dic[activity_tuple] = pd.DataFrame(network_property_dic[city][output_var][activity_tuple].values(), 
                                                  columns = ['spatial', activity_tuple[0], activity_tuple[1]],
                                                  index = network_property_dic[city][output_var][activity_tuple].keys())
    # normalize by using the spatial column (Q: Is this approach correct?)
    edge_bet_df_dic[activity_tuple][activity_tuple[0]] = edge_bet_df_dic[activity_tuple][activity_tuple[0]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple][activity_tuple[1]] = edge_bet_df_dic[activity_tuple][activity_tuple[1]]/edge_bet_df_dic[activity_tuple]['spatial']
    edge_bet_df_dic[activity_tuple]['spatial'] = np.ones(edge_bet_df_dic[activity_tuple].shape[0])
#     edge_bet_df_dic[activity_type]['spatial'] = edge_bet_df_dic[activity_type][activity_type]/edge_bet_df_dic[activity_type]['spatial']
    # print
    print(edge_bet_df_dic[activity_tuple].describe())


('strong', 'weak_sig_neg')
       spatial     strong  weak_sig_neg
count     10.0  10.000000     10.000000
mean       1.0   2.055356      1.418733
std        0.0   0.703378      0.155950
min        1.0   1.386671      1.155516
25%        1.0   1.631079      1.357860
50%        1.0   1.767687      1.407303
75%        1.0   2.356004      1.529094
max        1.0   3.645871      1.648447
('strong', 'weak_sig_pos')
       spatial     strong  weak_sig_pos
count     10.0  10.000000     10.000000
mean       1.0   2.106916      2.959454
std        0.0   0.504236      2.578534
min        1.0   1.537703      1.336763
25%        1.0   1.670490      1.816067
50%        1.0   2.042377      2.199208
75%        1.0   2.439080      2.371023
max        1.0   3.062907     10.065746
('weak_insig', 'weak_sig_neg')
       spatial  weak_insig  weak_sig_neg
count     10.0   10.000000     10.000000
mean       1.0    3.377131      2.947649
std        0.0    1.111717      1.496478
min        1.0    2.064658     

# Save